# Notebook for the Δ₄ amplitude computation  

## Part 1: from vertex amplitudes to the Δ₄ amplitude

In this Part we compute the Δ₄ amplitude starting from the vertex amplitudes tensors, 
which are already computed and stored in the current folder.

In order to compile this part it is only necessary to run the julia kernel (tested with julia 1.7.0)

In [1]:
# required pkgs  
using HalfIntegers
using JLD2   
using DelimitedFiles

In [2]:
current_folder = pwd()
vertex_ampls_folder = "$(current_folder)/vertex_ampls"

"/home/frisus95/Scrivania/Review_numerical_LQG/vertex_ampls"

In [3]:
# list of Immirzi parameters 
Immirzi_list = [0.1, 1.2]

# list of boundary spins (symmetric configurations)
j_boundary_list = [0.5,1]

# maximal number of shells 
Dl_max = 15

for Immirzi in Immirzi_list

for j_boundary in j_boundary_list    
        
   # if all boundary spins are equal to j, each boundary intertwiner has a range [0, 2j]     
   i_boundary_min = 0     
   i_boundary_max = convert(Int,2j_boundary) 
        
   # matrix in which the computed amplitudes will be stored         
   Delta_4_amplitudes = zeros(Dl_max + 1, i_boundary_max + 1)

      for i_boundary = i_boundary_min:i_boundary_max
          
         # julia arrays start at index 1   
         i_b = i_boundary + 1    
    
         for Dl = 0:Dl_max

            ampls_folder_source = vertex_ampls_folder*"/Immirzi=$(Immirzi)/all_j=$(j_boundary)/Dl=$(Dl)"       
                
            Delta_4_amp = 0.0 
                
               # if all boundary spins are equal to j, the bulk spin has a range [j, 3j]   
               j_bulk_min = j_boundary
               j_bulk_max = 3j_boundary  
    
               for j_bulk = j_bulk_min:j_bulk_max

               @load "$(ampls_folder_source)/j_bulk=$(j_bulk)_fulltensor.jld2" vertex 
                    
               # ranges of intertwiner space touched by the bulk spin      
               D = size(vertex[i_b,:,:,i_b,i_b])[1] # also [2] has the same range          
                    
               partial_amp = 0.0
  
               # when all spins are equal to j the phase is always unitary     
               for i_1234 in 1:D, i_1235 in 1:D, i_1236 in 1:D, i_1237 in 1:D
               partial_amp += vertex[i_b,i_1234,i_1235,i_b,i_b]*vertex[i_b,i_1235,i_1236,i_b,i_b]*
                              vertex[i_b,i_1236,i_1237,i_b,i_b]*vertex[i_b,i_1237,i_1234,i_b,i_b]     
               end     

               # dimension of bulk spin     
               partial_amp *= (2j_bulk + 1)
            
               Delta_4_amp += partial_amp  
                    
               end # j_bulk cycle
                
            Delta_4_amplitudes[Dl+1,i_b] = Delta_4_amp       
        
            # storing the computed amplitude in JLD2 format    
            Delta_4_JLD2_dest_folder = current_folder*"/Delta4_ampls/Immirzi=$(Immirzi)/all_j=$(j_boundary)/JLD2_format/all_i=$(i_boundary)/"
            mkpath(Delta_4_JLD2_dest_folder)         
            @save "$(Delta_4_JLD2_dest_folder)/Dl=$(Dl).jld2" Delta_4_amp
        
        end # shell cycle
            
        # storing the computed amplitudes in CSV format
        Delta_4_CSV_dest_folder = current_folder*"/Delta4_ampls/Immirzi=$(Immirzi)/all_j=$(j_boundary)/CSV_format" 
        mkpath(Delta_4_CSV_dest_folder)
        open("$(Delta_4_CSV_dest_folder)/Delta_4_ampls_Dlmax=$(Dl_max).csv", "w") do io
        writedlm(io, Delta_4_amplitudes, ',')
        end   
        
     end # i_boundary cycle
    
end # j_boundary cycle       

end # Immirzi cycle     
    

Notice that this way of contracting tensors in julia is neither efficient nor elegant, but we want to keep the code as clear as possible for those unfamiliar with julia's syntax


## Optional plotting with julia

In [4]:
# required pkgs
using Plots
using Plots.PlotMeasures
using LaTeXStrings

# set backend and style
pgfplotsx()
theme(:default)
default(markerstrokewidth = 0, 
        markerstrokealpha = 0, 
        linewidth = 2, 
        #grid=:none,   
        opacity = 0.8,
        gridlinewidth= 0.5,
        markersize = 10,
        markershape = :circle,
        tickfontsize = 18, 
        size = (900, 600), 
        yformatter = string,
        legendfontsize = 24, 
        guidefontsize=20,
        titlefontsize = 37,
        legend=(0.03,0.98), 
        xticks = 0:15, 
        foreground_color_axis = "black",
        foreground_color_border = "black",
        foreground_color_guide = "darkorange",
        foreground_color_text = "black",
        guidefontcolor = "black", 
        plot_titlefontcolor = "black",
        titlefontcolor = "black", 
        shape=[:circle])

Plots.PGFPlotsXBackend()

Plots.PGFPlotsXBackend()

In [5]:
# julia has still problems in plotting very small numbers, so we need to rescale them
# scientific format doesn't seem to work with Plots.jl
rescaling_factor = 1000000000000000000000000000000

for Immirzi in Immirzi_list
    
   j_boundary = 0.5
   Delta_4_CSV_format_folder = current_folder*"/Delta4_ampls/Immirzi=$(Immirzi)/all_j=$(j_boundary)/CSV_format" 
   Delta_4_ampls = readdlm("$(Delta_4_CSV_format_folder)/Delta_4_ampls_Dlmax=$(Dl_max).csv", ',') 
   Delta_4_ampls_rescaled = Delta_4_ampls*rescaling_factor  
             
   pl = plot(0:Dl_max, 
             [Delta_4_ampls_rescaled[:,1], Delta_4_ampls_rescaled[:,2]], 
             markersize = 10,
             markershape = :circle,
             palette = :RdYlGn_9,
             legend=(0.04,0.95),
             label=[L" i_b = 0 " L" i_b = 1 " ])
    xlabel!(L" \Delta l" )
    ylabel!(L" W " )
    title!(L"\mathrm{Delta\ 4\ amplitude\ (rescaled)\ j=%$(j_boundary)\ \gamma=%$(Immirzi) }\ ")
    savefig("$(current_folder)/Plots/Delta_4_j=$(j_boundary)_gamma=$(Immirzi).pdf")      
    
    
   j_boundary = 1.0
   Delta_4_CSV_format_folder = current_folder*"/Delta4_ampls/Immirzi=$(Immirzi)/all_j=$(j_boundary)/CSV_format" 
   Delta_4_ampls = readdlm("$(Delta_4_CSV_format_folder)/Delta_4_ampls_Dlmax=$(Dl_max).csv", ',') 
   Delta_4_ampls_rescaled = Delta_4_ampls*rescaling_factor  
             
   pl = plot(0:Dl_max, 
             [Delta_4_ampls_rescaled[:,1], Delta_4_ampls_rescaled[:,2], Delta_4_ampls_rescaled[:,3]], 
             markersize = 10,
             markershape = :circle,
             palette = :RdYlGn_9,
             legend=(0.04,0.95),
             label=[L" i_b = 0 " L" i_b = 1 " L" i_b = 2 "])
    xlabel!(L" \Delta l" )
    ylabel!(L" W " )
    title!(L"\mathrm{Delta\ 4\ amplitude\ (rescaled)\ j=%$(j_boundary)\ \gamma=%$(Immirzi) }\ ")
    savefig("$(current_folder)/Plots/Delta_4_j=$(j_boundary)_gamma=$(Immirzi).pdf")             
    
end # Immirzi cycle    

## Part 2: computing the vertex amplitudes

In this Part we compute the vertex amplitudes tensors required above. We employ the sl2cfoam-next library:
https://github.com/qg-cpt-marseille/sl2cfoam-next

See also:
https://arxiv.org/abs/2107.13952

In order to compile this part you first need to install the above library. 

In [3]:
# required packages  
using SL2Cfoam
using Dates

In [4]:
# print with flush
function log(x...)
    println(now(), " ", join(x, " ")...)
    flush(stdout)
end

log (generic function with 1 method)

In [5]:
function initializing_library(Immirzi)
   
folder = "/home/frisus95/Scrivania/sl2cfoam_next/data_sl2cfoam";
conf = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0);
SL2Cfoam.cinit(folder, Immirzi, conf);
    
end    

initializing_library (generic function with 1 method)

In [8]:
result_return = (ret = true, store = false, store_batches = false);

# list of Immirzi parameters 
Immirzi_list = [0.1, 1.2]

# list of boundary spins (symmetric configurations)
j_boundary_list = [0.5,1]

# maximal number of shells 
Dl_max = 15

for Immirzi in Immirzi_list
    
   initializing_library(Immirzi)    

   for j in j_boundary_list   
    
       for Dl = 0:Dl_max
        
          dest_folder = vertex_ampls_folder*"/Immirzi=$(Immirzi)/all_j=$(j)/Dl=$(Dl)"                 
          mkpath(dest_folder)
    
             # if all boundary spins are equal to j, bulk spin has range [j, 3j]   
             j_bulk_min = j
             j_bulk_max = 3j  
    
             for j_bulk = j_bulk_min:j_bulk_max
    
                v = vertex_compute([j, j, j, j, j, j, j, j_bulk, j, j], Dl; result = result_return);        
                log("computed full vertex tensor for Immirzi=$(Immirzi) j_boundary=$(j) j_bulk=$(j_bulk) Dl=$(Dl)")
                
                vertex = v.a;        
                @save "$(dest_folder)/j_bulk=$(j_bulk)_fulltensor.jld2" vertex         
               
             end # bulk spin cycle
      
        end # shell cycle
            
    end # j_boundary cycle
    
end  # Immirzi cycle

2022-01-10T21:13:00.082 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Dl=0
2022-01-10T21:13:02.800 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Dl=0
2022-01-10T21:13:02.880 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Dl=1
2022-01-10T21:13:02.903 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Dl=1
2022-01-10T21:13:02.916 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Dl=2
2022-01-10T21:13:02.954 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Dl=2
2022-01-10T21:13:03.046 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Dl=3
2022-01-10T21:13:03.070 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Dl=3
2022-01-10T21:13:03.104 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=0.5 Dl=4
2022-01-10T21:13:03.148 computed full vertex tensor for Immirzi=0.1 j_boundary=0.5 j_bulk=1.5 Dl=4
2022-01-10